In [2]:
#import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")

In [3]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables

In [4]:
#load cost data
con_generation = CSV.File("../data/conventional_generators.csv") |> DataFrame

#load wind_technicaldata
wind_generation = CSV.File("../data/wind_farms.csv") |> DataFrame

# load wind profile
wind_profile = CSV.File("../data/wind_powerprofile_200.csv") |> DataFrame

print(con_generation)

12×11 DataFrame
 Row │ Conventional generators  Location [node]  Production cost [$/MWh]  Upward reserve cost [$/MW]  Downward reserve cost [$/MW]  Capacity [MW]  Maximum upward reserve provision capability [MW]  Maximum downward reserve provision capability [MW]  RUi (MW/h)_x  RDi (MW/h)_x  P inii (MW)_x 
     │ Int64                    Int64            Float64                  Float64                     Float64                       Int64          Int64                                             Int64                                               Int64         Int64         Int64         
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │                       1                1                    13.32                        1.68        

In [5]:
# load equilibrium and market results from Task1 result foler
equilibrium = CSV.File("../Task 1/results/equilibrium_model1.csv") |> DataFrame
market = CSV.File("../Task 1/results/market_clearing_model1.csv") |> DataFrame

#get hour from market df
hour = equilibrium[!, :hour][1]

market_price = equilibrium[!, :market_price][1]


# get all the columns, which are starting with x_con and x_wind
x_con_names = [col for col in names(market) if occursin("x_con", col)]
x_wind_names = [col for col in names(market) if occursin("x_wind", col)]
y_names = [col for col in names(market) if occursin("y", col)]

x_con = market[!, x_con_names]
x_wind = market[!, x_wind_names]
y = market[!, y_names]


print(x_con)

1×12 DataFrame
 Row │ x_con1   x_con2   x_con3   x_con4   x_con5   x_con6   x_con7   x_con8   x_con9   x_con10  x_con11  x_con12 
     │ Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64  Float64 
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │     0.0      0.0      0.0      0.0      0.0    155.0    155.0    400.0    400.0    300.0    310.0  147.466

In [6]:
# market changes defintion
import Random
Random.seed!(1234)

# make a list of all the positive x_con values and their x_con_list_indexindex
x_con_list = []
x_con_list_index = []
for i in 1:size(x_con, 1)
    for j in 1:size(x_con, 2)
        if x_con[i, j] > 0
            push!(x_con_list, x_con[i, j])
            push!(x_con_list_index, (i, j))
        end
    end
end

# return random number between 1 and length of x_con_list
random_number = rand(1:length(x_con_list))
# get the index of the x_con_list_indexindex
outage_index = x_con_list_index[random_number]

outage_number = 8
# give the column name of the x_con
outage_generator = x_con_names[outage_number]

# get wind generatorion differences
x_wind_bal = []

for i in 1:2
    push!(x_wind_bal, x_wind[1,i] *0.1)#0.1
end

for i in 3:4
    push!(x_wind_bal, -x_wind[1,i] * 0.15)#0.15
end

balance_amount = x_con[1, outage_number] + sum(x_wind_bal)

382.32161524527373

In [7]:
#demand bids
name = "demand_hour_0.csv"
demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame
 
# number of convential generators
G = size(con_generation, 1)

# number of demand
D = size(demand_bids, 1)

# number of wind generators
W = size(wind_generation, 1)

4

In [8]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], objective_value = Float64[])
equilibrium_df = DataFrame(hour = Int[], market_price_bal = Float64[], market_quantity = Float64[], market_state = String[])

# For x variables, manually add each column. This is a one-time setup.
for i in 1:G
    result_df[!, Symbol("x_con_down$i")] = Float64[]
end

for i in 1:G
    result_df[!, Symbol("x_con_up$i")] = Float64[]
end

# For y variables, manually add each column. This is a one-time setup.
for i in 1:D
    result_df[!, Symbol("y_down$i")] = Float64[]
end

In [9]:
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for every generator
unregister(model, :x)  # Unregister the existing variable named "x" from the model

#demand bids
name = "demand_hour_" * string(hour-1) * ".csv"
demand_bids = CSV.File("../data/demand_bids_hour/" * name) |> DataFrame

# number of demand
D = size(demand_bids, 1)

#@variable(model, x_bin[1:G], Bin)  # binary variable for on/off status
@variable(model, x_con_down[1:G])  # power output variable

#@variable(model, x_bin[1:G], Bin)  # binary variable for on/off status
@variable(model, x_con_up[1:G])  # power output variable

# Create explicit constraints for the bounds, to be able to access their dual values later
lb_g = Vector{Any}(undef, G+G)
ub_g = Vector{Any}(undef, G+G)
for g in 1:G
    lb_g[g] = @constraint(model, x_con_down[g] >= 0)
    ub_g[g] = @constraint(model, x_con_down[g] <= con_generation[g,8])
end

for g in 1:G
    lb_g[g+G] = @constraint(model, x_con_up[g] >= 0)
    ub_g[g+G] = @constraint(model, x_con_up[g] <= con_generation[g,7])
end

#constraint for generators including day-ahead market
for g in 1:G
    @constraint(model, x_con_down[g] <= x_con[1, g])
    @constraint(model, x_con_up[g] + x_con[1, g] <= con_generation[g,6])
end

# extra constraint for the outage generator
@constraint(model, x_con_down[outage_number] == 0)
@constraint(model, x_con_up[outage_number] == 0)



# decision variables for demand
@variable(model, y_down[1:size(demand_bids, 1)])  # Create the new variable
# Create explicit constraints for the bounds, to be able to access their dual values later
lb_d = Vector{Any}(undef, D)
ub_d = Vector{Any}(undef, D)
for d in 1:D
    lb_d[d] = @constraint(model, y_down[d] >= 0)
    ub_d[d] = @constraint(model, y_down[d] <= y[1,d])
end

# Add the constraint for the balance between supply and demand
@constraint(model, BalanceConstraint, sum(x_con_up[g] for g in 1:G)  + sum(y_down[d] for d in 1:D) - sum(x_con_down[g] for g in 1:G) == balance_amount)

# Define the objective function
@objective(model, Min, sum((market_price+0.1*con_generation[g,3])*x_con_up[g] for g in 1:G) + 400 * sum(y_down[d] for d in 1:D) - sum((market_price-0.13*con_generation[g,3])*x_con_down[g] for g in 1:G))

# Print the model
#print(model)

# Solve the model
optimize!(model)

# Check the status of the solution
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Hour ", hour, ": Optimal solution found")
    #println("Objective value: ", objective_value(model))
    #println("x values: ", [value(x[i]) for i in 1:G])
    #println("x_wind values: ", [value(x_wind[i]) for i in 1:W])
    #println("y values: ", [value(y[i]) for i in 1:D])
    println(dual(BalanceConstraint))

    market_state = 0

    if balance_amount > 0
        market_state = "Deficit"
    else
        market_state = "Excess"
    end
    #add dual values to dataframe
    push!(equilibrium_df, [hour, dual(BalanceConstraint), balance_amount, market_state])

    #save results to dataframe
    push!(result_df, [hour, objective_value(model), [value(x_con_down[i]) for i in 1:G]..., [value(x_con_up[i]) for i in 1:G]..., [value(y_down[i]) for i in 1:D]...])

elseif status == MOI.INFEASIBLE
    println("Hour ", hour, ": No feasible solution found")
else
    println("Hour ", hour, ": Solver status: ", status)
end


Hour 14: Optimal solution found
13.501000000000001


Row,hour,objective_value,x_con_down1,x_con_down2,x_con_down3,x_con_down4,x_con_down5,x_con_down6,x_con_down7,x_con_down8,x_con_down9,x_con_down10,x_con_down11,x_con_down12,x_con_up1,x_con_up2,x_con_up3,x_con_up4,x_con_up5,x_con_up6,x_con_up7,x_con_up8,x_con_up9,x_con_up10,x_con_up11,x_con_up12,y_down1,y_down2,y_down3,y_down4,y_down5,y_down6,y_down7,y_down8,y_down9,y_down10,y_down11,y_down12,y_down13,y_down14,y_down15,y_down16,y_down17
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,14,4867.41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,40.0,70.0,180.0,12.3216,0.0,0.0,0.0,0.0,0.0,5.68434e-14,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
print(market_price)

10.89

In [11]:
equilibrium_df

Row,hour,market_price_bal,market_quantity,market_state
,Int64,Float64,Float64,String
1,14,13.501,382.322,Deficit


In [12]:
#Access the dual values of all the generators constraints and add them to a new dataframe, in scientific notation
duals_g = DataFrame(hour = Int[], gen = Symbol[], lower_bound = Float64[], upper_bound = Float64[])
for i in 1:G
    push!(duals_g, [hour, Symbol("x_con_down$i"), dual(lb_g[i]), dual(ub_g[i])])
end
for i in 1:G
    push!(duals_g, [hour, Symbol("x_con_up$i"), dual(lb_g[i+G]), dual(ub_g[i+G])])
end

duals_d = DataFrame(hour = Int[], node = Int[],lower_bound = Float64[], upper_bound = Float64[])
for i in 1:D
    push!(duals_d, [hour, demand_bids[i,1], dual(lb_d[i]), dual(ub_d[i])])
end


#save results to dataframe
CSV.write("results/optimization_results_generation5.csv", duals_g)
CSV.write("results/optimization_results_demand5.csv", duals_d)


"results/optimization_results_demand5.csv"

In [13]:
result_df

Row,hour,objective_value,x_con_down1,x_con_down2,x_con_down3,x_con_down4,x_con_down5,x_con_down6,x_con_down7,x_con_down8,x_con_down9,x_con_down10,x_con_down11,x_con_down12,x_con_up1,x_con_up2,x_con_up3,x_con_up4,x_con_up5,x_con_up6,x_con_up7,x_con_up8,x_con_up9,x_con_up10,x_con_up11,x_con_up12,y_down1,y_down2,y_down3,y_down4,y_down5,y_down6,y_down7,y_down8,y_down9,y_down10,y_down11,y_down12,y_down13,y_down14,y_down15,y_down16,y_down17
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,14,4867.41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,40.0,70.0,180.0,12.3216,0.0,0.0,0.0,0.0,0.0,5.68434e-14,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
#save result_df to csv
CSV.write("results/market_clearing_model5.csv", result_df)
CSV.write("results/equilibrium_model5.csv", equilibrium_df)

"results/equilibrium_model5.csv"

In [15]:
# sign = 0
# if (x_con[1, outage_index[2]] + sum(x_wind_bal)) > 0
#     #deficit
#     sign = -1
# else
#     #excess
#     sign = 1
# end



profit_df_one_price = DataFrame()

# insert the assigned capacity for each generator
energy_day_ahead = []
for i in 1:G
    # make array with the assigned capacity
    push!(energy_day_ahead, x_con[1,i])
end
profit_df_one_price[!, :energy_day_ahead] = energy_day_ahead

#calculate the profit for each generator
day_ahead_revenue = []
for i in 1:G
    push!(day_ahead_revenue,x_con[1,i]* market_price)
end
profit_df_one_price[!, :day_ahead_revenue] = day_ahead_revenue

capacity_change = []
for i in 1:G
    if i == outage_number
        push!(capacity_change, -1*x_con[1, outage_number])
    else
        push!(capacity_change, 0)
    end
end
profit_df_one_price[!, :capacity_change] = capacity_change

#insert column for penalty for each generator
penalty = []
for i in 1:G
    push!(penalty, dual(BalanceConstraint) * profit_df_one_price[!, :capacity_change][i] * -1)
end
profit_df_one_price[!, :penalty] = penalty

# insert the assigned capacity for each generator
balance_volume = []
for i in 1:G
    # make array with the assigned capacity
    push!(balance_volume, value(x_con_up[i]) - value(x_con_down[i]))
end
profit_df_one_price[!, :balance_volume] = balance_volume

#calculate the profit for each generator
balance_revenue = []
for i in 1:G
    push!(balance_revenue, (value(x_con_up[i]) - value(x_con_down[i])) * dual(BalanceConstraint))
end
profit_df_one_price[!, :balance_revenue] = balance_revenue

#total revenue for each generator
profit_df_one_price[!, :total_revenue] = profit_df_one_price[!, :day_ahead_revenue] + profit_df_one_price[!, :balance_revenue] - profit_df_one_price[!, :penalty]

#insert the variable costs for each generator
variable_costs = []
for i in 1:G
    push!(variable_costs, con_generation[i,3])
end
profit_df_one_price[!, :variable_costs] = variable_costs

#calculate costs for each generator
profit_df_one_price[!, :costs] = (profit_df_one_price[!, :energy_day_ahead] + profit_df_one_price[!, :capacity_change] + profit_df_one_price[!, :balance_volume]) .* profit_df_one_price[!, :variable_costs]

#calculate the profit for each generator
profit_df_one_price[!, :profit] = profit_df_one_price[!, :total_revenue]-profit_df_one_price[!, :costs]

print(profit_df_one_price)


CSV.write("results/profits_model5.csv", profit_df_one_price)

#pretty_table(profit_df_one_price, backend = Val(:latex))

12×10 DataFrame
 Row │ energy_day_ahead  day_ahead_revenue  capacity_change  penalty  balance_volume  balance_revenue  total_revenue  variable_costs  costs     profit    
     │ Any               Any                Any              Any      Any             Any              Float64        Any             Float64   Float64   
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ 0.0               0.0                0                -0.0     40.0            540.04                 540.04   13.32            532.8        7.24
   2 │ 0.0               0.0                0                -0.0     40.0            540.04                 540.04   13.32            532.8        7.24
   3 │ 0.0               0.0                0                -0.0     70.0            945.07                 945.07   20.7            1449.0     -503.93
   4 │ 0.0               0.0                0               

"results/profits_model5.csv"

In [16]:
#do the same for thw wind farms
profit_df_one_price_wind = DataFrame()

# insert the assigned capacity for each generator
energy_day_ahead = []
for i in 1:W
    # make array with the assigned capacity
    push!(energy_day_ahead, x_wind[1,i])
end
profit_df_one_price_wind[!, :energy_day_ahead] = energy_day_ahead

#calculate the profit for each generator
day_ahead_revenue = []
for i in 1:W
    push!(day_ahead_revenue,x_wind[1,i]* market_price)
end
profit_df_one_price_wind[!, :day_ahead_revenue] = day_ahead_revenue

capacity_change = []
for i in 1:W
    push!(capacity_change, x_wind_bal[i]*-1)
end
profit_df_one_price_wind[!, :capacity_change] = capacity_change

#insert column for penalty for each generator
penalty = []
for i in 1:W
        push!(penalty, dual(BalanceConstraint) * profit_df_one_price_wind[!, :capacity_change][i] * -1)
end
profit_df_one_price_wind[!, :penalty] = penalty

balance_volume = []
for i in 1:W
    # make array with the assigned capacity
    push!(balance_volume, 0)
end
profit_df_one_price_wind[!, :balance_volume] = balance_volume

#calculate the profit for each generator
balance_revenue = []
for i in 1:W
    push!(balance_revenue, 0)
end
profit_df_one_price_wind[!, :balance_revenue] = balance_revenue

#total revenue for each generator
profit_df_one_price_wind[!, :total_revenue] = profit_df_one_price_wind[!, :day_ahead_revenue] + profit_df_one_price_wind[!, :balance_revenue] - profit_df_one_price_wind[!, :penalty]

#insert the variable costs for each generator
variable_costs = []
for i in 1:W
    push!(variable_costs, 0)
end
profit_df_one_price_wind[!, :variable_costs] = variable_costs

#calculate costs for each generator
profit_df_one_price_wind[!, :costs] = (profit_df_one_price_wind[!, :energy_day_ahead] + profit_df_one_price_wind[!, :capacity_change] + profit_df_one_price_wind[!, :balance_volume]) .* profit_df_one_price_wind[!, :variable_costs]

#calculate the profit for each generator
profit_df_one_price_wind[!, :profit] = profit_df_one_price_wind[!, :total_revenue]-profit_df_one_price_wind[!, :costs]

print(profit_df_one_price_wind)

4×10 DataFrame
 Row │ energy_day_ahead  day_ahead_revenue  capacity_change  penalty   balance_volume  balance_revenue  total_revenue  variable_costs  costs    profit   
     │ Any               Any                Any              Any       Any             Any              Float64        Any             Float64  Float64  
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ 147.45            1605.73            -14.745          199.072   0               0                     1406.66   0                   0.0  1406.66
   2 │ 99.6239           1084.9             -9.96239         134.502   0               0                      950.402  0                   0.0   950.402
   3 │ 137.481           1497.17            20.6221          -278.419  0               0                     1775.58   0                   0.0  1775.58
   4 │ 145.091           1580.04            21.7637          -293.

In [17]:
balance_amount

382.32161524527373

In [18]:
state = 0
if balance_amount > 0
     #deficit
    state = -1
else
     #excess
    state = 1
end



profit_df_two_price = DataFrame()

# insert the assigned capacity for each generator
energy_day_ahead = []
for i in 1:G
    # make array with the assigned capacity
    push!(energy_day_ahead, x_con[1,i])
end
profit_df_two_price[!, :energy_day_ahead] = energy_day_ahead

#calculate the profit for each generator
day_ahead_revenue = []
for i in 1:G
    push!(day_ahead_revenue,x_con[1,i]* market_price)
end
profit_df_two_price[!, :day_ahead_revenue] = day_ahead_revenue

capacity_change = []
for i in 1:G
    if i == outage_number
        push!(capacity_change, -1*x_con[1, outage_number])
    else
        push!(capacity_change, 0)
    end
end
profit_df_two_price[!, :capacity_change] = capacity_change

#insert column for penalty for each generator
penalty = []
for i in 1:G
    if (state * profit_df_two_price[!, :capacity_change][i]) > 0
        push!(penalty, dual(BalanceConstraint) * profit_df_two_price[!, :capacity_change][i] * -1)
    else
        push!(penalty, market_price * profit_df_two_price[!, :capacity_change][i] * -1)
    end
end
profit_df_two_price[!, :penalty] = penalty

# insert the assigned capacity for each generator
balance_volume = []
for i in 1:G
    # make array with the assigned capacity
    push!(balance_volume, value(x_con_up[i]) - value(x_con_down[i]))
end
profit_df_two_price[!, :balance_volume] = balance_volume

#calculate the profit for each generator
balance_revenue = []
for i in 1:G
    push!(balance_revenue, (value(x_con_up[i]) - value(x_con_down[i])) * dual(BalanceConstraint))
end
profit_df_two_price[!, :balance_revenue] = balance_revenue

#total revenue for each generator
profit_df_two_price[!, :total_revenue] = profit_df_two_price[!, :day_ahead_revenue] + profit_df_two_price[!, :balance_revenue] - profit_df_two_price[!, :penalty]

#insert the variable costs for each generator
variable_costs = []
for i in 1:G
    push!(variable_costs, con_generation[i,3])
end
profit_df_two_price[!, :variable_costs] = variable_costs

#calculate costs for each generator
profit_df_two_price[!, :costs] = (profit_df_two_price[!, :energy_day_ahead] + profit_df_two_price[!, :capacity_change] + profit_df_two_price[!, :balance_volume]) .* profit_df_two_price[!, :variable_costs]

#calculate the profit for each generator
profit_df_two_price[!, :profit] = profit_df_two_price[!, :total_revenue]-profit_df_two_price[!, :costs]

print(profit_df_two_price)


CSV.write("results/profits_model5.csv", profit_df_two_price)

#pretty_table(profit_df_two_price, backend = Val(:latex))

12×10 DataFrame
 Row │ energy_day_ahead  day_ahead_revenue  capacity_change  penalty  balance_volume  balance_revenue  total_revenue  variable_costs  costs     profit    
     │ Any               Any                Any              Any      Any             Any              Float64        Any             Float64   Float64   
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ 0.0               0.0                0                -0.0     40.0            540.04                 540.04   13.32            532.8        7.24
   2 │ 0.0               0.0                0                -0.0     40.0            540.04                 540.04   13.32            532.8        7.24
   3 │ 0.0               0.0                0                -0.0     70.0            945.07                 945.07   20.7            1449.0     -503.93
   4 │ 0.0               0.0                0               

"results/profits_model5.csv"

In [19]:
#do the same for thw wind farms
profit_df_two_price_wind = DataFrame()

# insert the assigned capacity for each generator
energy_day_ahead = []
for i in 1:W
    # make array with the assigned capacity
    push!(energy_day_ahead, x_wind[1,i])
end
profit_df_two_price_wind[!, :energy_day_ahead] = energy_day_ahead

#calculate the profit for each generator
day_ahead_revenue = []
for i in 1:W
    push!(day_ahead_revenue,x_wind[1,i]* market_price)
end
profit_df_two_price_wind[!, :day_ahead_revenue] = day_ahead_revenue

capacity_change = []
for i in 1:W
    push!(capacity_change, x_wind_bal[i]*-1)
end
profit_df_two_price_wind[!, :capacity_change] = capacity_change

#insert column for penalty for each generator
penalty = []
for i in 1:W
    if (state * profit_df_two_price_wind[!, :capacity_change][i]) > 0
        push!(penalty, dual(BalanceConstraint) * profit_df_two_price_wind[!, :capacity_change][i] * -1)
    else
        push!(penalty, market_price * profit_df_two_price_wind[!, :capacity_change][i] * -1)
    end
end
profit_df_two_price_wind[!, :penalty] = penalty

balance_volume = []
for i in 1:W
    # make array with the assigned capacity
    push!(balance_volume, 0)
end
profit_df_two_price_wind[!, :balance_volume] = balance_volume

#calculate the profit for each generator
balance_revenue = []
for i in 1:W
    push!(balance_revenue, 0)
end
profit_df_two_price_wind[!, :balance_revenue] = balance_revenue

#total revenue for each generator
profit_df_two_price_wind[!, :total_revenue] = profit_df_two_price_wind[!, :day_ahead_revenue] + profit_df_two_price_wind[!, :balance_revenue] - profit_df_two_price_wind[!, :penalty]

#insert the variable costs for each generator
variable_costs = []
for i in 1:W
    push!(variable_costs, 0)
end
profit_df_two_price_wind[!, :variable_costs] = variable_costs

#calculate costs for each generator
profit_df_two_price_wind[!, :costs] = (profit_df_two_price_wind[!, :energy_day_ahead] + profit_df_two_price_wind[!, :capacity_change] + profit_df_two_price_wind[!, :balance_volume]) .* profit_df_two_price_wind[!, :variable_costs]

#calculate the profit for each generator
profit_df_two_price_wind[!, :profit] = profit_df_two_price_wind[!, :total_revenue]-profit_df_two_price_wind[!, :costs]

print(profit_df_two_price_wind)

4×10 DataFrame
 Row │ energy_day_ahead  day_ahead_revenue  capacity_change  penalty   balance_volume  balance_revenue  total_revenue  variable_costs  costs    profit   
     │ Any               Any                Any              Any       Any             Any              Float64        Any             Float64  Float64  
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ 147.45            1605.73            -14.745          199.072   0               0                     1406.66   0                   0.0  1406.66
   2 │ 99.6239           1084.9             -9.96239         134.502   0               0                      950.402  0                   0.0   950.402
   3 │ 137.481           1497.17            20.6221          -224.575  0               0                     1721.74   0                   0.0  1721.74
   4 │ 145.091           1580.04            21.7637          -237.